# Imports

In [46]:
import pandas as pd 
import numpy as np
import math

#%pip install unidecode
import unidecode

import warnings
warnings.filterwarnings('ignore')

## Data Links

- Position Players: https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=c,3,4,6,7,8,9,10,11,12,23,13,14,15,16,17,18,19,20,34,35,36,37,38,39,41,42,43,44,45,48,49,47,205,206,207,208,209,210,211,212,50,40,51,52,61,65,58,62,63,64,102,103,104,105,106,107,109,110,200,21,22,201,60,111&season=2022&month=0&season1=2010&ind=1&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=

- Pitchers: https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=c,3,4,5,7,8,9,10,114,11,12,13,55,57,14,15,16,17,18,19,20,21,22,23,28,24,25,26,27,43,41,42,6,332,117,45,62,118,119,59,63,64,65,123,218,219,220,221,222,223,225,330,113,217,331,36,37,39,40,66,122,51,44,46,47,48,49,50,52,53,105,106,107,108,109,110,111&season=2022&month=0&season1=2010&ind=1&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=

- Defense: https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=c,6,7,8,9,28,39,61,59,17,18&season=2022&month=0&season1=2010&ind=1&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=

In [47]:
# Importing scraped/collected data
arb11 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2011')
arb12 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2012')
arb13 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2013')
arb14 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2014')
arb15 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2015')
arb16 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2016')
arb17 = pd.read_excel('Data/collected_arb_data.xlsx', sheet_name ='arb2017')

# Importing found data
arb18 = pd.read_csv('Data/arb2018.csv')
arb19 = pd.read_csv('Data/arb2019.csv')
arb20 = pd.read_csv('Data/arb2020.csv')
arb21 = pd.read_csv('Data/arb2021.csv')
arb22 = pd.read_csv('Data/arb2022.csv')
arb23 = pd.read_csv('Data/arb2023.csv')

# Importing Fangraphs Data
fg_hit = pd.read_csv('Data/HittingData.csv')
fg_pitch = pd.read_csv('Data/PitchingData.csv')
fg_def = pd.read_csv('Data/FieldingData.csv')

# Data Cleaning

## Helper Functions

In [48]:
'''
Goal: Clean either found data or collected arbitration data
--------------------------------------------------------------------------------
Inputs: df (DataFrame of imported and read csv files)
        year (int, year of arbitration)

Returns: df (DataFrame with cleaning done)
'''
def clean_arb_data(df, year):
    # Set arbitration dataset year variable to the previous year
    #print(df.iloc[1,1], type(df.iloc[1,1]))
    #prev_year = int(df.columns[0].split(' ')[1]) - 1
    prev_year = year - 1
    
    # Convert first row into the column names
    df.columns = df.iloc[0, :]

    # Return where the actual data starts (3rd row of Google Sheet)
    df = df.iloc[2:, :]

    # Rename columns of interest
    df.rename(columns = {df.columns[0]: 'Name',
                         df.columns[1]: 'Prev. Season',
                         df.columns[3]: 'Position',
                         df.columns[4]: 'Service Time',
                         str(year) + ' Salary': 'Salary'}, inplace = True)


    # Filter out all players cut, demoted, or signed to extension
    df = df[df['Salary'].notnull() & df['Salary'].str.startswith("$")]

    #Add Season column, with the year set back one
    #to indicate the year that affected the next year's arbitration salary
    df['Prev. Season'] = prev_year

    # Change Service time to the year
    df = df.astype({"Service Time": str})
    df["Service Time"] = df['Service Time'].apply(lambda x: x[0])

    df = df[['Name', 'Prev. Season', 'Position','Service Time','Salary']].reset_index(drop = True)

    # Changes all of Salary column values from strings to integers
    df['Salary'] = df['Salary'].apply(lambda x:  int(float(x.replace('$', '').replace('M', ''))*(10**6)) if x.__contains__('MM') else int(x.replace('$', '').replace(',','')))

    # Applies standardize_pos function to all values in Position column
    df["Position"] = df['Position'].apply(standard_position)

    # Applies standardize_names function to all values in Player column
    df['Name'] = df['Name'].apply(standard_name)

    #print(df.shape)
    return df

In [49]:
'''
Goal: Standardize names of strings to match the names of other datasets
--------------------------------------------------------------------------------
Inputs: inp_str (string representing name of MLB player)

Returns: The name reconstituted from its old format into a new one,
removing accents and other special characters not found in other datasets
'''
def standard_name(inp_str):
    #print(inp_str)

    # Splits original name string and concatenates it in reverse order, with
    # unicode function applied
    if inp_str.__contains__(', '):
        name_li = inp_str.split(', ')
        out_str = unidecode.unidecode(name_li[1] + ' ' + name_li[0])
    else:
        out_str = unidecode.unidecode(inp_str)

    # Replace any special character phrases
    out_str = out_str.replace(' ,Adeg','').replace(' ,A+','')

    return out_str

In [50]:
'''
Goal: Standardize positions of players to develop discrete categories
  e.g. LHP-S is a left-handed starting pitcher, or SP
--------------------------------------------------------------------------------
Inputs: inp_str (string representing position of a given MLB player)

Returns: Player's position string reformatted to
'''
def standard_position(inp_str):
    out_str = inp_str.upper()

    # Splits the formatting of old string into SP (starting pitcher) or RP (relief pitcher)
    if out_str.__contains__('LHP') or out_str.__contains__('RHP'):
        if out_str[-1] == 'S':
            out_str = 'SP'
        else:
            out_str = 'RP'

    # Case of players who play multiple positions, which will have their
    # first listed position
    if out_str.__contains__('-'):
        # Special case of 'C-OF'
        if out_str[0] == 'C':
            out_str = out_str[0]
        else:
            out_str = out_str[0:2]

    # position best matching INF of all other positions is 2B
    if out_str == 'INF':
        out_str = '2B'
    return out_str

In [51]:
'''
Goal: Convert all percentage statistics in Fangraphs datasets to floats
  e.g. '34.6%' is converted to 34.6 (float)
--------------------------------------------------------------------------------
Inputs: val (string the percentage statistic we want to convert)

Returns: new_per, converted % string into a float value
'''
def remove_percentages(val):
    if type(val) != float and val.__contains__("%"):
        new_per = float(val[:-1])
        return new_per


In [52]:
'''
Goal: Convert all IP values that end with '.1' and '.2' to '
  .33' and '.67' respectively
  e.g. 123.2 (float) is converted to 123.67 (float)
--------------------------------------------------------------------------------
Inputs: val (float indicating IP for pitcher)

Returns: new_IP, converted from float to string, then back to float value
'''
def change_IP(val):
    
    if math.isnan(val):
        val = 0.0
        
    val = str(val)

    if val.endswith(".2"):
        # Change .2 IP to 2/3 IP
        
        new_IP = val.replace('.2','.67')
        return float(new_IP)
    elif  val.endswith(".1"):
        # Change .1 IP to 1/3 IP
        
        new_IP = val.replace('.1', '.33')
        return float(new_IP)
    else:
        return float(val)

In [53]:
df_pitchers[df_pitchers.Name == 'Shohei Ohtani']
fg_hit[fg_hit.Name == 'Shohei Ohtani']

,Season,Name,Team,Age,G,PA,H,1B,2B,3B,...,F-Strike%,SwStr%,wSB,SB,CS,UBR,Spd,BsR,playerid,mlbamid
594,2021,Shohei Ohtani,LAA,26,158,639,138,58,26,8,...,57.1%,15.0%,0.5,26,10,0.4,6.9,2.5,19755,660271
645,2018,Shohei Ohtani,LAA,23,114,367,93,48,21,2,...,58.3%,13.0%,0.2,10,4,-0.3,5.5,1.1,19755,660271
1000,2022,Shohei Ohtani,LAA,27,157,666,160,90,30,6,...,59.8%,12.6%,-2.0,11,9,0.2,4.7,0.3,19755,660271
1565,2019,Shohei Ohtani,LAA,24,106,425,110,67,20,5,...,58.4%,12.3%,0.9,12,3,-0.9,6.2,1.4,19755,660271
5222,2020,Shohei Ohtani,LAA,25,46,175,29,16,6,0,...,52.6%,13.3%,0.8,7,1,0.8,5.2,2.1,19755,660271


## Clean Data

In [54]:
# Clean all data
arb11 = clean_arb_data(arb11, 2011)
arb12 = clean_arb_data(arb12, 2012)
arb13 = clean_arb_data(arb13, 2013)
arb14 = clean_arb_data(arb14, 2014)
arb15 = clean_arb_data(arb15, 2015)
arb16 = clean_arb_data(arb16, 2016)
arb17 = clean_arb_data(arb17, 2017)
arb18 = clean_arb_data(arb18, 2018)
arb19 = clean_arb_data(arb19, 2019)
arb20 = clean_arb_data(arb20, 2020)
arb21 = clean_arb_data(arb21, 2021)
arb22 = clean_arb_data(arb22, 2022)
arb23 = clean_arb_data(arb23, 2023)

In [55]:
# Add together all arbitration data
arb_dfs = [arb11, arb12, arb13, arb14, arb15, arb16, arb17, arb18, arb19, arb20, arb21, arb22, arb23]

df = pd.concat(arb_dfs, ignore_index=True)
print(df.shape)
df.head(10)

(2362, 5)


,Name,Prev. Season,Position,Service Time,Salary
0,Adam Jones,2010,CF,3,3250000
1,Alberto Callaspo,2010,3B,3,2000000
2,Alberto Gonzalez,2010,2B,2,600000
3,Alex Gordon,2010,LF,3,1400000
4,Alexi Casilla,2010,SS,3,865000
5,Andres Torres,2010,CF,3,2100000
6,Andy Sonnanstine,2010,SP,3,912000
7,Angel Pagan,2010,OF,4,3500000
8,Anibal Sanchez,2010,SP,4,3700000
9,Armando Galarraga,2010,SP,2,2300000


In [56]:
#Splits large into dataframe containing PITCHERS ONLY
df_pitchers = df.loc[df['Position'].isin(['RP','SP'])].reset_index()

# Fix name differences for the join performed later

df_pitchers = df_pitchers.replace({'J.C. Ramirez':'JC Ramirez',
                                   'Carl Edwards':'Carl Edwards Jr.',
                                   'Wily * Peralta':'Wily Peralta',
                                   'Kenyan Middleton':'Keynan Middleton',
                                   'Matt Boyd':'Matthew Boyd',
                                   'Vincent Velasquez':'Vince Velasquez',
                                   'Austin L. Adams':'Austin Adams',
                                   'Zach Britton': 'Zack Britton',
                                   'Anthony DeSclafini': "Anthony DeSclafani",
                                   'A.J. Ramos': 'AJ Ramos',
                                   'Daniel Winkler': 'Dan Winkler',
                                   'Nate Eovaldi': 'Nathan Eovaldi',
                                   'Michael Dunn': 'Mike Dunn',
                                   'Robbie Ross': 'Robbie Ross Jr.',
                                   'Rubby De La Rosa': 'Rubby de la Rosa',
                                   'Michael Fiers': 'Mike Fiers',
                                   'Nate Karns': 'Nathan Karns',
                                    'Juan Carlos Oviedo': 'Juan Oviedo',
                                   'Jose O. Berrios': 'Jose Berrios'})


# Delete position players who pitched
df_pitchers = df_pitchers[~df_pitchers["Name"].isin(['Tony Wolters', 'Josh Phegley'])]

# Splits large df into dataframe containing POSITION PLAYERS ONLY
df_plyrs = df.loc[~df['Position'].isin(['RP','SP'])].reset_index()
# Fix name differences for the join performed later
df_plyrs = df_plyrs.replace({"Dan Vogelbach": 'Daniel Vogelbach',
                             "Clint Frazier": "Jackson Frazier",
                             "Javy Baez" : "Javier Baez",
                             "Delino DeShields Jr." : "Delino DeShields",
                             "Albert Almora" : "Albert Almora Jr.",
                             "Jacoby Jones" : "JaCoby Jones",
                             "Giovanny Urshela" : "Gio Urshela",
                             "Michael Taylor" : "Michael A. Taylor",
                             "Vladimir Jr. Guerrero" : "Vladimir Guerrero Jr.",
                             "Teoscar Hernández" : "Teoscar Hernandez",
                             "Steven Souza": "Steven Souza Jr.",
                             'Nicholas Castellanos': 'Nick Castellanos',
                             'Jr. Melvin Upton': 'Melvin Upton Jr.',
                             'Mike Morse': 'Michael Morse',
                             'Alejandro de Aza': 'Alejandro De Aza',
                             'Eric Young': 'Eric Young Jr.',
                             'Dee Gordon': 'Dee Strange-Gordon',
                             "Travis D'Arnaud": "Travis d'Arnaud",
                             'D.J. LeMahieu': 'DJ LeMahieu',
                             'Ronnie Torreyes': 'Ronald Torreyes',
                             'Yu-Cheng Chang': 'Yu Chang',
                             'Rafael Oretga': 'Rafael Ortega',
                             'LaMonte Wade': 'LaMonte Wade Jr.',
                             'Vladimir Jr. Guerrero': 'Vladimir Guerrero Jr.',
                              'Giovanny Urshela': 'Gio Urshela'})

In [57]:
#For all Fangraphs data, drop Fangraphs, MLB playerids and Teams
# Rename Season column to Prev. Season to be joined
fg_drop_cols = ['playerid', 'Team','mlbamid']

fg_hit = fg_hit.drop(fg_drop_cols, axis=1)
fg_hit = fg_hit.rename(columns={'Season': 'Prev. Season'})

# Editing name in both Fangraphs and Cot's Dataset due to discrepancies
fg_hit['Name'] = fg_hit['Name'].replace({'Cedric Mullins II':'Cedric Mullins'})


fg_pitch = fg_pitch.drop(fg_drop_cols, axis=1)
fg_pitch = fg_pitch.rename(columns={'Season': 'Prev. Season'})

fg_def = fg_def.drop(fg_drop_cols + ['Pos'], axis=1)
# Editing name in both Fangraphs and Cot's Dataset due to discrepancies
fg_def['Name'] = fg_def["Name"].replace({'Cedric Mullins II': 'Cedric Mullins'})

fg_def = fg_def.rename(columns={'Season': 'Prev. Season'})

# Add metrics of players who played different positions in a single season
fg_def = fg_def.groupby(['Prev. Season','Name']).sum()

# Remove groupby into a pivot table, preserving order of columns
fg_def = fg_def.pivot_table(index=['Name','Prev. Season']).reset_index()[['Name','Prev. Season', 'Inn','E','DRS','UZR','RAA','FRM','SB','CS']]

# Standardize innings played on defense
fg_def['Inn'] = fg_def['Inn'].apply(change_IP)

In [58]:
ohtani = df_pitchers[df_pitchers.Name == 'Shohei Ohtani'].replace('SP','DH').replace(2138, 2362)
df_plyrs.loc[1023] = [i for i in ohtani.values[0]]
df_plyrs.tail()

,index,Name,Prev. Season,Position,Service Time,Salary
1019,2348,Danny Jansen,2022,C,4,3500000
1020,2354,Daulton Varsho,2022,C,2,3050000
1021,2360,Lane Thomas,2022,RF,3,2200000
1022,2361,Ildemaro Vargas,2022,SS,3,975000
1023,2362,Shohei Ohtani,2022,DH,5,30000000


# Merge Data

In [59]:
# Merge arb data for pitchers with Fangraphs data
merged_pitchers = pd.merge(df_pitchers, fg_pitch, on=['Name', 'Prev. Season'], how='left').drop(['index'], axis = 1)

# Change IP figures
merged_pitchers['IP'] = merged_pitchers['IP'].apply(change_IP)
merged_pitchers['Start-IP'] = merged_pitchers['Start-IP'].apply(change_IP)
merged_pitchers['Relief-IP'] = merged_pitchers['Relief-IP'].apply(change_IP)

# Apply the remove_percentages function to all percentage (string) columns
for i in ['K-BB%','CSW%','SwStr%','Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
       'TTO%', 'CStr%','HR/FB','LOB%','LD%', 'GB%', 'FB%','IFFB%', 'IFH%', 'BUH%',
          'O-Swing%','Z-Swing%','Swing%','O-Contact%','Z-Contact%','Contact%','Zone%']:
    merged_pitchers[i] = merged_pitchers[i].apply(remove_percentages)

# Fill-in all pitchers who missed a year during arbitration due to injury/demotion
merged_pitchers.fillna(dict.fromkeys(merged_pitchers.columns[5:], 0), inplace=True)

merged_pitchers.head()

,Name,Prev. Season,Position,Service Time,Salary,Age,W,L,G,GS,...,IFFB%,IFH%,BUH%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%
0,Andy Sonnanstine,2010,SP,3,912000,27.0,3.0,1.0,41.0,4.0,...,16.5,7.5,50.0,27.5,66.9,47.0,64.6,88.7,81.5,49.4
1,Anibal Sanchez,2010,SP,4,3700000,26.0,13.0,12.0,32.0,32.0,...,9.9,9.1,16.7,29.5,64.0,46.5,65.1,86.1,79.3,49.3
2,Armando Galarraga,2010,SP,2,2300000,28.0,4.0,9.0,25.0,24.0,...,9.3,6.1,66.7,28.2,67.2,45.9,71.2,91.9,84.9,45.3
3,Bill Bray,2010,RP,3,645000,27.0,0.0,2.0,35.0,0.0,...,11.6,3.6,0.0,32.9,66.7,46.9,54.9,81.1,70.4,41.7
4,Boone Logan,2010,RP,3,1200000,25.0,2.0,0.0,51.0,0.0,...,2.5,8.0,100.0,27.9,72.3,47.7,48.0,85.3,73.2,44.5


In [60]:
# Merge past arbitration position players with season statistics
merged_plyrs = pd.merge(df_plyrs, fg_hit, on=['Name', 'Prev. Season'], how='left').drop(['index'], axis = 1)
merged_plyrs = merged_plyrs.rename(columns={"SB": "SB_run", "CS": "CS_run"})

merged_plyrs = pd.merge(merged_plyrs, fg_def, on=['Name', 'Prev. Season'], how='left')
merged_plyrs = merged_plyrs.rename(columns={"SB": "SB_def", "CS": "CS_def"})

# Apply the remove_percentages function to all percentage columns
for i in ['BB%','K%','LD%', 'GB%', 'FB%', 'IFH%', 'BUH%',
       'HR/FB','Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%','F-Strike%',
       'TTO%','O-Swing%','Z-Swing%','Swing%','O-Contact%','Z-Contact%','Contact%','SwStr%']:
    merged_plyrs[i] = merged_plyrs[i].apply(remove_percentages)

# Fill-in all pitchers who missed a year during arbitration due to injury/demotion
merged_plyrs.fillna(dict.fromkeys(merged_plyrs.columns[5:], 0), inplace=True)

merged_plyrs.head()

,Name,Prev. Season,Position,Service Time,Salary,Age,G,PA,H,1B,...,Spd,BsR,Inn,E,DRS,UZR,RAA,FRM,SB_def,CS_def
0,Adam Jones,2010,CF,3,3250000,24.0,149.0,621.0,165.0,116.0,...,4.5,-0.3,1298.33,7.0,-6.0,-3.8,0.0,0.0,0.0,0.0
1,Alberto Callaspo,2010,3B,3,2000000,27.0,146.0,601.0,149.0,110.0,...,3.5,-5.5,1236.00,11.0,4.0,3.4,0.0,0.0,0.0,0.0
2,Alberto Gonzalez,2010,2B,2,600000,27.0,115.0,198.0,46.0,37.0,...,2.8,-2.7,393.60,5.0,7.0,7.4,0.0,0.0,0.0,0.0
3,Alex Gordon,2010,LF,3,1400000,26.0,74.0,281.0,52.0,34.0,...,2.5,-4.9,598.33,7.0,-3.0,-3.2,0.0,0.0,0.0,0.0
4,Alexi Casilla,2010,SS,3,865000,25.0,69.0,170.0,42.0,30.0,...,8.0,0.7,370.33,5.0,2.0,3.0,0.0,0.0,0.0,0.0


# Add Previous Salary

In [61]:
# Create a new DataFrame with all years for each 'Unique_ID'
all_years_pitchers = pd.DataFrame({
    'Prev. Season': merged_pitchers.groupby('Name')['Prev. Season'].apply(lambda x: range(x.min(), x.max() + 1))
}).explode('Prev. Season')

# Merge the original DataFrame with the new DataFrame to fill missing years
merged_pitchers = merged_pitchers.merge(all_years_pitchers, on=['Prev. Season', 'Name'], how='right')

# Sort the DataFrame by 'Unique_ID' and 'Year'
merged_pitchers.sort_values(['Name', 'Prev. Season'], inplace=True)

# Fill missing Data_Point values using the previous year's value
merged_pitchers['Prev. Salary'] = merged_pitchers.groupby('Name')['Salary'].shift()

merged_pitchers

,Name,Prev. Season,Position,Service Time,Salary,Age,W,L,G,GS,...,IFH%,BUH%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,Prev. Salary
0,A.J. Griffin,2016.0,SP,4,2000000.0,28.0,7.0,4.0,23.0,23.0,...,4.0,0.0,27.0,65.7,44.1,68.8,84.9,79.4,44.1,NaN
1,A.J. Minter,2020.0,RP,2,1300000.0,26.0,1.0,1.0,22.0,0.0,...,4.0,0.0,31.0,74.4,49.4,64.8,76.0,71.9,42.3,NaN
2,A.J. Minter,2021.0,RP,3,2200000.0,27.0,3.0,6.0,61.0,0.0,...,9.5,50.0,34.2,67.2,49.0,53.9,81.4,70.9,44.9,1300000.0
3,A.J. Minter,2022.0,RP,4,4287500.0,28.0,5.0,4.0,75.0,0.0,...,3.3,33.3,38.3,69.9,51.4,57.6,78.8,69.5,41.3,2200000.0
4,AJ Ramos,2015.0,RP,3,3400000.0,28.0,2.0,4.0,71.0,0.0,...,2.9,0.0,34.0,66.2,46.7,44.6,82.4,65.7,39.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,Zack Britton,2016.0,RP,4,11400000.0,28.0,2.0,1.0,69.0,0.0,...,3.9,50.0,37.0,64.6,47.6,42.0,83.1,63.5,38.6,6750000.0
1380,Zack Britton,2017.0,RP,5,12000000.0,29.0,2.0,1.0,38.0,0.0,...,6.1,0.0,31.8,66.0,45.5,57.1,87.1,74.5,40.0,11400000.0
1381,Zack Wheeler,2016.0,SP,3,800000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1382,Zack Wheeler,2017.0,SP,4,1900000.0,27.0,3.0,7.0,17.0,17.0,...,7.4,40.0,24.8,69.1,46.1,65.2,86.1,80.3,48.1,800000.0


In [62]:
# Create a new DataFrame with all years for each 'Unique_ID'
all_years_plyrs = pd.DataFrame({
    'Prev. Season': merged_plyrs.groupby('Name')['Prev. Season'].apply(lambda x: range(x.min(), x.max() + 1))
}).explode('Prev. Season')

# Merge the original DataFrame with the new DataFrame to fill missing years
merged_plyrs = merged_plyrs.merge(all_years_plyrs, on=['Prev. Season', 'Name'], how='right')

# Sort the DataFrame by 'Unique_ID' and 'Year'
merged_plyrs.sort_values(['Name', 'Prev. Season'], inplace=True)

# Fill missing Data_Point values using the previous year's value
merged_plyrs['Prev. Salary'] = merged_plyrs.groupby('Name')['Salary'].shift()

merged_plyrs 

,Name,Prev. Season,Position,Service Time,Salary,Age,G,PA,H,1B,...,BsR,Inn,E,DRS,UZR,RAA,FRM,SB_def,CS_def,Prev. Salary
0,A.J. Ellis,2012.0,C,2,2000000.0,31.0,133.0,505.0,114.0,80.0,...,-3.3,1151.00,6.0,-6.0,0.0,0.0,-9.1,74.0,36.0,NaN
1,A.J. Ellis,2013.0,C,3,3550000.0,32.0,115.0,448.0,93.0,65.0,...,-0.4,972.33,3.0,-4.0,0.0,0.0,-2.3,35.0,28.0,2000000.0
2,A.J. Ellis,2014.0,C,4,4250000.0,33.0,93.0,347.0,54.0,42.0,...,-8.0,773.67,4.0,-6.0,0.0,0.0,-14.0,48.0,16.0,3550000.0
3,A.J. Ellis,2015.0,C,5,4500000.0,34.0,63.0,217.0,43.0,27.0,...,-2.7,492.00,0.0,-5.0,0.0,0.0,-8.5,22.0,18.0,4250000.0
4,A.J. Pollock,2017.0,CF,5,7750000.0,29.0,112.0,466.0,113.0,60.0,...,2.6,873.33,1.0,6.0,1.3,1.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,Yonder Alonso,2016.0,1B,5,4000000.0,29.0,156.0,532.0,122.0,81.0,...,-2.8,1174.33,5.0,-4.0,0.8,1.0,0.0,0.0,0.0,2650000.0
1051,Yunel Escobar,2010.0,SS,3,2900000.0,27.0,135.0,567.0,127.0,104.0,...,-0.8,1179.33,18.0,9.0,4.1,0.0,0.0,0.0,0.0,NaN
1052,Zack Cozart,2014.0,SS,3,2350000.0,28.0,147.0,543.0,112.0,85.0,...,3.0,1274.33,10.0,16.0,13.0,0.0,0.0,0.0,0.0,NaN
1053,Zack Cozart,2015.0,SS,4,2925000.0,29.0,53.0,214.0,50.0,30.0,...,0.5,449.67,3.0,4.0,1.0,0.0,0.0,0.0,0.0,2350000.0


In [16]:
# All players not tendered a contract through arbitration for a specific year
# Non-tendered by team and signing as a free agent OR re-entering arbitration after an extension

print('Number of Total majoirty NaN rows (Pitchers):', merged_pitchers.Position.isna().sum())
merged_pitchers[merged_pitchers.Position.isna()].head()

Number of Total majoirty NaN rows (Pitchers): 44


,Name,Prev. Season,Position,Service Time,Salary,Age,W,L,G,GS,...,IFH%,BUH%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,Prev. Salary
113,Blake Wood,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,560000.0
114,Blake Wood,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,Brandon Kintzler,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1075000.0
182,Buck Farmer,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1850000.0
206,Carl Edwards Jr.,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500000.0


In [17]:
# All pitchers not tendered a contract through arbitration for a specific year
# Non-tendered by team and signing as a free agent OR re-entering arbitration after an extension
print('Number of Total majoirty NaN rows (Position Players):', merged_plyrs.Position.isna().sum())

merged_pitchers[merged_pitchers.Position.isna()].head(10)

Number of Total majoirty NaN rows (Position Players): 30


,Name,Prev. Season,Position,Service Time,Salary,Age,W,L,G,GS,...,IFH%,BUH%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,Prev. Salary
113,Blake Wood,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,560000.0
114,Blake Wood,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,Brandon Kintzler,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1075000.0
182,Buck Farmer,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1850000.0
206,Carl Edwards Jr.,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500000.0
207,Carl Edwards Jr.,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,Carl Edwards Jr.,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,Clayton Richard,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5240000.0
280,Clayton Richard,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,Dan Winkler,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610000.0


In [63]:
# Delete majority NaN rows from dataset
merged_plyrs = merged_plyrs[merged_plyrs.Position.isna() == False]
merged_pitchers = merged_pitchers[merged_pitchers.Position.isna() == False]

# Splitting Data, Train and Test

In [64]:
train_pitchers = merged_pitchers[merged_pitchers['Prev. Season'] != 2022].reset_index()
train_pitchers.head()

,index,Name,Prev. Season,Position,Service Time,Salary,Age,W,L,G,...,IFH%,BUH%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,Prev. Salary
0,0,A.J. Griffin,2016.0,SP,4,2000000.0,28.0,7.0,4.0,23.0,...,4.0,0.0,27.0,65.7,44.1,68.8,84.9,79.4,44.1,NaN
1,1,A.J. Minter,2020.0,RP,2,1300000.0,26.0,1.0,1.0,22.0,...,4.0,0.0,31.0,74.4,49.4,64.8,76.0,71.9,42.3,NaN
2,2,A.J. Minter,2021.0,RP,3,2200000.0,27.0,3.0,6.0,61.0,...,9.5,50.0,34.2,67.2,49.0,53.9,81.4,70.9,44.9,1300000.0
3,4,AJ Ramos,2015.0,RP,3,3400000.0,28.0,2.0,4.0,71.0,...,2.9,0.0,34.0,66.2,46.7,44.6,82.4,65.7,39.5,NaN
4,5,AJ Ramos,2016.0,RP,4,6550000.0,29.0,1.0,4.0,67.0,...,5.1,50.0,27.3,61.0,41.9,49.7,83.8,71.2,43.2,3400000.0


In [65]:
# Test set for pitchers
test_pitchers = merged_pitchers[merged_pitchers['Prev. Season'] == 2022].reset_index()
test_pitchers.head()

,index,Name,Prev. Season,Position,Service Time,Salary,Age,W,L,G,...,IFH%,BUH%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,Prev. Salary
0,3,A.J. Minter,2022.0,RP,4,4287500.0,28.0,5.0,4.0,75.0,...,3.3,33.3,38.3,69.9,51.4,57.6,78.8,69.5,41.3,2200000.0
1,7,Aaron Civale,2022.0,RP,3,2600000.0,27.0,5.0,6.0,20.0,...,5.2,100.0,33.7,66.6,46.7,64.5,88.0,77.7,39.4,NaN
2,18,Adam Cimber,2022.0,RP,4,3150000.0,31.0,10.0,6.0,77.0,...,8.0,16.7,32.1,71.1,50.6,73.3,86.6,82.2,47.5,1575000.0
3,31,Adrian Houser,2022.0,SP,4,3600000.0,29.0,6.0,10.0,22.0,...,6.4,100.0,29.6,66.2,43.0,78.1,90.4,85.0,36.5,2425000.0
4,32,Adrian Morejon,2022.0,RP,3,800000.0,23.0,5.0,1.0,26.0,...,27.3,0.0,33.0,76.4,53.7,68.8,82.7,78.3,47.8,NaN


In [68]:
train_plyrs = merged_plyrs[merged_plyrs['Prev. Season'] != 2022].reset_index()
train_plyrs.head()

,index,Name,Prev. Season,Position,Service Time,Salary,Age,G,PA,H,...,BsR,Inn,E,DRS,UZR,RAA,FRM,SB_def,CS_def,Prev. Salary
0,0,A.J. Ellis,2012.0,C,2,2000000.0,31.0,133.0,505.0,114.0,...,-3.3,1151.00,6.0,-6.0,0.0,0.0,-9.1,74.0,36.0,NaN
1,1,A.J. Ellis,2013.0,C,3,3550000.0,32.0,115.0,448.0,93.0,...,-0.4,972.33,3.0,-4.0,0.0,0.0,-2.3,35.0,28.0,2000000.0
2,2,A.J. Ellis,2014.0,C,4,4250000.0,33.0,93.0,347.0,54.0,...,-8.0,773.67,4.0,-6.0,0.0,0.0,-14.0,48.0,16.0,3550000.0
3,3,A.J. Ellis,2015.0,C,5,4500000.0,34.0,63.0,217.0,43.0,...,-2.7,492.00,0.0,-5.0,0.0,0.0,-8.5,22.0,18.0,4250000.0
4,4,A.J. Pollock,2017.0,CF,5,7750000.0,29.0,112.0,466.0,113.0,...,2.6,873.33,1.0,6.0,1.3,1.0,0.0,0.0,0.0,NaN


In [66]:
test_plyrs = merged_plyrs[merged_plyrs['Prev. Season'] == 2022].reset_index()
test_plyrs.head()

,index,Name,Prev. Season,Position,Service Time,Salary,Age,G,PA,H,...,BsR,Inn,E,DRS,UZR,RAA,FRM,SB_def,CS_def,Prev. Salary
0,12,Abraham Toro,2022.0,3B,2,1250000.0,25.0,109.0,352.0,60.0,...,-1.1,599.33,6.0,0.0,-2.7,0.0,0.0,0.0,0.0,NaN
1,15,Adalberto Mondesi,2022.0,SS,5,3045000.0,26.0,15.0,54.0,7.0,...,1.0,131.00,0.0,1.0,1.4,3.0,0.0,0.0,0.0,3000000.0
2,53,Alex Verdugo,2022.0,OF,4,6300000.0,26.0,152.0,644.0,166.0,...,-1.1,1305.30,5.0,-5.0,-0.9,-4.0,0.0,0.0,0.0,3550000.0
3,59,Amed Rosario,2022.0,SS,5,7800000.0,26.0,153.0,670.0,180.0,...,1.5,1286.00,14.0,4.0,6.3,-5.0,0.0,0.0,0.0,4950000.0
4,67,Andrew Knizner,2022.0,C,3,1100000.0,27.0,97.0,293.0,56.0,...,-1.8,692.30,6.0,-9.0,0.0,0.0,-8.7,21.0,7.0,NaN


In [69]:
train_plyrs = train_plyrs[train_plyrs['Prev. Season'] != 2010]
train_pitchers = train_pitchers[train_pitchers['Prev. Season'] != 2010]

In [70]:
print("Training and Validation Data (2010-2022 Hitters)", train_plyrs.shape)
print("Training and Validation Data (2010-2022 Pitchers)", train_pitchers.shape)

print("")
print("Test Data (2023 Hitters)", test_plyrs.shape)
print("Training and Validation Data (2023 Pitchers)", test_pitchers.shape)

Training and Validation Data (2010-2022 Hitters) (875, 79)
Training and Validation Data (2010-2022 Pitchers) (1120, 83)

Test Data (2023 Hitters) (89, 79)
Training and Validation Data (2023 Pitchers) (140, 83)


In [71]:
train_plyrs.to_csv('arb_train_plyrs.csv', index=False)
train_pitchers.to_csv('arb_train_pitchers.csv', index=False)
test_plyrs.to_csv('arb_test_plyrs.csv', index=False)
test_pitchers.to_csv('arb_test_pitchers.csv', index=False)